In [1]:
%matplotlib inline
from collections import Counter
from collections import defaultdict
import scanpy as sc
import scrublet as scr
import pandas as pd
import pickle as pkl
import numpy as np
from bbknn import bbknn
import scipy
import matplotlib.pyplot as plt
import re
import glob
import os
import sys
from geosketch import gs
from numpy import cov
import scipy.cluster.hierarchy as spc
import seaborn as sns; sns.set(color_codes=True)
from sklearn.linear_model import LogisticRegression
import sklearn
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.set_figure_params(dpi=80, color_map='viridis')
sc.logging.print_versions()

/home/jovyan/my-conda-envs/workhorse/lib/python3.6/site-packages/anndata/_core/anndata.py:21: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import RangeIndex


scanpy==1.4.4.post1 anndata==0.7.1 umap==0.3.10 numpy==1.18.1 scipy==1.3.0 pandas==1.0.3 scikit-learn==0.22.2.post1 statsmodels==0.11.1 python-igraph==0.8.0 louvain==0.6.1


/home/jovyan/my-conda-envs/workhorse/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.cluster.k_means_ module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.cluster. Anything that cannot be imported from sklearn.cluster is now part of the private API.
  warnings.warn(message, FutureWarning)


In [16]:
#Introduce variables

#name of first object (arbitrary)
data1 = "_Training"
Object1 = './overall_data/Healthy_all_data.h5ad'
#provide categorical containing donor information
donor_id = 'donor_id'
PCs = [5,10,15,20,25]
#provide cateorical to join between datasets, this shoould be annotations for cells (obs col)
cat1 = 'final'
#provide an output path and a folder name to be created
output = "./logit_regression_out_PCA_var_v3/"

## Start of processing module

In [3]:
# Load dataset 1
adata_orig = sc.read(Object1)

In [4]:
adata_orig.obs.columns

Index(['sample_id', 'mad_prd', 'Status', 'Site', 'Site_old', 'Tissue',
       'Enrichment', 'Location', 'donor_id', 'Sex', 'Age', 'stage',
       'anno_final', 'final'],
      dtype='object')

In [ ]:
#insert pre-calculated umap coordinates
umap_coord = np.load('./healthy_data_umap.npy')
adata_orig.obsm["X_umap"] = umap_coord

## Correcting cell labels

In [5]:
sc_var = cat1
#Remove all "NANS" from the annotations
adata_orig = adata_orig[adata_orig.obs[~adata_orig.obs[sc_var].isna()].index]

#Get the current annotation
clusters = list(adata_orig.obs[sc_var].unique())
clusters

#List corrections
celltypes = [
'Post_proliferation_KC',
 'Post_proliferation_KC',
 'Post_proliferation_KC',
 'C_Melanocyte',
 'Post_proliferation_KC',
 'Post_proliferation_KC',
 'Pre_proliferation_KC',
 'Proliferating_KC',
 'Pre_proliferation_KC',
 'LE2',
 'F2',
 'LE1',
 'F1',
 'LC_2',
 'Th',
 'F3',
 'VE1',
 'VE2',
 'LC_3',
 'Tc',
 'LC_1',
 'Treg',
 'NK',
 'Macro_1',
 'ILC1_NK',
 'ILC2_3',
 'LC_4',
 'ILC1',
 'Macro_2',
 'Plasma',
 'DC2',
 'Inf_mono',
 'Mono',
 'moDC_3',
 'moDC_1',
 'moDC_2',
 'MigDC',
 'DC1',
 'Pericyte_2_inflamm',
 'Pericyte_1_inflamm',
 'Schwaan_2_non_myelinating',
 'VE3',
 'Schwaan_1_Stroma_Schwan_myelinating',
 'M_A_Mast_cell'
]
cell_dict = dict(zip(clusters, celltypes))
adata_orig.obs[sc_var] = adata_orig.obs[sc_var].map(cell_dict)

Trying to set attribute `.obs` of view, copying.


# LR function

In [6]:
def LR_compare(adata, train_x ,train_label,subset_predict, subset_train,penalty='l2',sparcity=0.2,col_name='predicted'):
    #Define LR parameters
    penalty='l2'
    sparcity=0.2
    lr = LogisticRegression(penalty=penalty, C=sparcity)

    if train_x == 'X':
        train_label = adata.obs[common_cat].values
        train_label = train_label[subset_train]
        train_x = adata.X
        predict_x = train_x
        train_x = train_x[subset_train, :]
        predict_x = train_x
        predict_x = predict_x[subset_predict]


    elif train_x in adata.obsm.keys():
        #Define training parameters
        train_label = adata.obs[common_cat].values
        train_label = train_label[subset_train]
        train_x = adata.obsm[train_x]
        predict_x = train_x
        train_x = train_x[subset_train, :]

        #Define Prediction parameters
        predict_x = predict_x[subset_predict]
        predict_x = pd.DataFrame(predict_x)
        predict_x.index = adata.obs[subset_predict].index

        
    #Train predictive model
    model = lr.fit(train_x, train_label)
    lr.fit(train_x, train_label)
    predict = lr.predict_proba(predict_x)

    #Create prediction table and map to adata.obs
    predict = lr.predict(predict_x)
    predict = pd.DataFrame(predict)
    predict.index = adata.obs[subset_predict].index
    adata.obs[col_name] = adata.obs.index
    adata.obs[col_name] = adata.obs[col_name].map(predict[0])

# Start of predictive and output loop

In [7]:
os.chdir('/home/jovyan/data/projects/HCA_skin_project/')
print(os.getcwd())

/home/jovyan/data/projects/HCA_skin_project


In [8]:
for i in PCs:
    print(i)

5
10
15
20
25


In [19]:
#inte = [5,10,15,20,25,30,35,40,45,50,55]
inte = [60,65,70]

In [ ]:
os.chdir('/home/jovyan/data/projects/HCA_skin_project/')
print(os.getcwd())

#Create output directory
if os.path.exists(output) == True:
    print("Path already exists!")
else:
    os.mkdir(output)

os.chdir('./'+output)
print(os.getcwd())

#Start of the subset and plotting loop
for i in inte:
    adata = adata_orig[:]
    adata2 = adata[:]
    #adata2 = adata2[~adata2.obs[donor_id].isin([i])]
#name of second object
    data2 = "_prediction"
#provide cateorical to join between datasets
    cat2 = cat1
    
#create a common obs column in both datasets containing the data origin tag
    common_cat = "corr_concat" 
    adata.obs[common_cat] = adata.obs[cat1].astype(str) + data1
    adata2.obs[common_cat] = adata2.obs[cat2].astype(str) + data2
    adata.obs = adata.obs.astype('category')
    adata2.obs = adata2.obs.astype('category')
    #adata.raw = adata
    #adata2.raw = adata2

#concat the data
    concat = adata.concatenate(adata2, join='inner', index_unique='-', batch_key='Status')
    adata = concat
    print('data_concatenated!')
#Get PCA by covarainces of variable genes
    sc.pp.normalize_per_cell(adata, counts_per_cell_after=1e4)
    sc.pp.log1p(adata)
    sc.pp.highly_variable_genes(adata, min_mean=0.1, max_mean=4)

#%% PCA
    sc.pp.pca(adata, n_comps=i, use_highly_variable=True, svd_solver='arpack')
    
#Define the seprator category in the column of interest, this works by partial matches and enables a-symmetric comparisons
    Data1_group = data1
    Data2_group = data2

#Define the common .obs column between cancatinated data
    common_cat = "corr_concat"

#Define the resource to train and predict on, PCA or X or UMAP (#if you wish to use gene expression, train_x = 'X' or 'X_pca' , or 'X_umap'
    #train_x = 'X'
    train_x = 'X_pca'
    ########################################################################################
    group1 = (adata.obs[common_cat][adata.obs[common_cat].str.contains(Data1_group)]).unique()
    group1 = list(group1)
    group2 = (adata.obs[common_cat][adata.obs[common_cat].str.contains(Data2_group)]).unique()
    group2 = list(group2)
    subset_predict = np.array(adata.obs[common_cat].isin(group2))
    subset_train = np.array(adata.obs[common_cat].isin(group1))
    train_label = adata.obs[common_cat].values
    #########################################################################################
    
#Run LR
    LR_compare(adata, train_x,train_label,subset_predict, subset_train,penalty='l2',sparcity=0.2,col_name='predicted')
    print('LR_predictions completed for loop ' + str(i) + '!' )
#Plot predictions in probability heatmap
    x='predicted'
    y = common_cat
    y_attr = adata.obs[y]
    x_attr = adata.obs[x]
    crs_tbl = pd.crosstab(x_attr, y_attr)
    #crs_tbl_test = crs_tbl
    
#Cross table removes all result which have 0 matches, add these back into table
    vals = list(crs_tbl.index)
    pred = pd.DataFrame(group1)
    missing_vals = pred.iloc[:,0][~(pred.iloc[:,0].isin(vals))]
    crs_tbl = crs_tbl.T
    for missing in missing_vals:
        crs_tbl[missing] = 0
    crs_tbl = crs_tbl.reindex(sorted(crs_tbl.columns), axis=1)
    crs_tbl = crs_tbl.T
    crs_tbl = crs_tbl.reindex(sorted(crs_tbl.columns), axis=1)
    for col in crs_tbl :
        crs_tbl[col] = crs_tbl[col].div(crs_tbl[col].sum(axis=0)).multiply(100)

#Optionally save this image!
    #plot_df_heatmap(crs_tbl.T, cmap='coolwarm', rotation=90, figsize=figsize, vmin=20, vmax=70)
    #pal = sns.diverging_palette(240, 10, n=10)
    #g = sns.clustermap(crs_tbl.T, cmap=pal,vmin=20, vmax=70,linewidths=.5)
    #g = sns.heatmap(crs_tbl, cmap=pal, vmin=0, vmax=100,linewidths=.5 ,center=50,square=True )
    #plt.show();
    
#Save probability distribution
    prob_out_name = "./logist_prediction_prob_donor_minus_" + str(i)+ ".csv"
    crs_tbl.to_csv(prob_out_name)
    print("LR probs saved for loop " + str(i) +"!")
############################

    #Crete a holder for current obj
    adata3 = adata[:]
    #load original preiction object
    adata_predicted = adata2[:]
    #Assign matches
    adata3 = adata3[(adata3.obs[common_cat].isin(group2))]
    adata3.obs[common_cat].unique()
    adata_predicted.obs['predicted'] = adata_predicted.obs.index
    adata3.obs.index = adata_predicted.obs.index
    adata3.obs[common_cat].astype(str)
    adata_predicted.obs['predicted'] = adata3.obs["predicted"].astype(str)
    
#Frequency redistribution by additive assignment if reclustering is done
    cluster_prediction = "clus_prediction"
    #clusters_reassign = "leiden_res5"
    #res= 5
    #lr_predicted_col = 'predicted'
    
##Plot Umap with Rand index and mutual info score
    #sc.pp.neighbors(adata_predicted,n_neighbors=15, n_pcs=i)
    #sc.pp.highly_variable_genes(adata_predicted, min_mean=0.1, max_mean=4)
    sc.pp.pca(adata_predicted, n_comps=i, use_highly_variable=False, svd_solver='arpack')
    sc.external.pp.bbknn(adata_predicted, batch_key='donor_id', approx=True, metric='angular', copy=False, n_pcs=i, trim=None, n_trees=10, use_faiss=True, set_op_mix_ratio=1.0, local_connectivity=1)
    sc.tl.umap(adata_predicted)

    #sc.tl.leiden(adata_predicted, resolution= res, key_added= clusters_reassign, random_state=24, n_iterations=-1)
    #adata_predicted.obs[cluster_prediction] = adata_predicted.obs.index
    #for z in adata_predicted.obs[clusters_reassign].unique():
    #    df = adata_predicted.obs
    #    df = df[(df[clusters_reassign].isin([z]))]
    #    df_count = pd.DataFrame(df[lr_predicted_col].value_counts())
    #    freq_arranged = df_count.index
    #    cat = freq_arranged[0]
    #    df.loc[:,cluster_prediction] = cat
    #    adata_predicted.obs.loc[adata_predicted.obs[clusters_reassign] == z, [cluster_prediction]] = cat
    adata_predicted.obs[cluster_prediction] = adata_predicted.obs['predicted']
#caculate Rand and MI
    rand=sklearn.metrics.adjusted_rand_score(list(adata_predicted.obs[cat1]), list(adata_predicted.obs[cluster_prediction]))
    mi=sklearn.metrics.adjusted_mutual_info_score(list(adata_predicted.obs[cat1]), list(adata_predicted.obs[cluster_prediction]), average_method='arithmetic')
#include info in plot title
##Edit the predicted col and get all missing values
    adata_predicted.obs[cluster_prediction] = adata_predicted.obs[cluster_prediction].str.replace('_Training', '', regex=True)
    adata_predicted.obs[cluster_prediction]
    fig_name = str(i) + " Adj_Rnd= " + str(rand) + " Mut_info= " + str(mi)
    sc.pl.umap(adata_predicted,color=cluster_prediction,title = fig_name,save=("_"+str(i)+".png"))
    print("Umap, rand,Mi plotted for loop " + str(i) + "!")
# prints the missing and additional elements in list2 into our dataframe
    missing = set(list(adata_predicted.obs[cat1])).difference(list(adata_predicted.obs[cluster_prediction]))

##Create data frame and populate with scoring metrics
    temp_score = pd.DataFrame(columns=[str(i)],dtype=object)
    temp_score = pd.DataFrame(temp_score.T)
    temp_score["adj_rand"] = rand
    temp_score["mutual_info"] = mi
    temp_score["missing_vals"] = "NAN"
    temp_score.at[str(i), "missing_vals"] = missing

    if 'concat_score' in globals():
        concat_score = pd.concat([concat_score, temp_score])
    else:
        concat_score = temp_score[:]
    
#Save file?
    #save_file_predicted = "./adata_predicted_donor_minus_" + i
    #adata.write(save_file_orig)
    
#Return resources back to system and restart the loop
    #gc.collect()
    del adata2
    del adata3
    print("end of loop!")
#Write out overall scored output
concat_score.to_csv("./concatenated_scores_minus_PCs.csv")

/home/jovyan/data/projects/HCA_skin_project
Path already exists!
/home/jovyan/data/projects/HCA_skin_project/logit_regression_out_PCA_var_v3


Trying to set attribute `.obs` of view, copying.
Trying to set attribute `.obs` of view, copying.


data_concatenated!
normalizing by total count per cell
    finished (0:00:24): normalized adata.X and added    'n_counts', counts per cell before normalization (adata.obs)
extracting highly variable genes
    finished (0:00:47)
--> added
    'highly_variable', boolean vector (adata.var)
    'means', float vector (adata.var)
    'dispersions', float vector (adata.var)
    'dispersions_norm', float vector (adata.var)
computing PCA with n_comps = 60
computing PCA on highly variable genes
    finished (0:00:56)


/home/jovyan/my-conda-envs/workhorse/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/jovyan/my-conda-envs/workhorse/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/s

LR_predictions completed for loop 60!
LR probs saved for loop 60!


...

data_concatenated!
normalizing by total count per cell
    finished (0:00:15): normalized adata.X and added    'n_counts', counts per cell before normalization (adata.obs)
extracting highly variable genes
    finished (0:00:46)
--> added
    'highly_variable', boolean vector (adata.var)
    'means', float vector (adata.var)
    'dispersions', float vector (adata.var)
    'dispersions_norm', float vector (adata.var)
computing PCA with n_comps = 65
computing PCA on highly variable genes
    finished (0:01:05)


/home/jovyan/my-conda-envs/workhorse/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/jovyan/my-conda-envs/workhorse/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/s

LR_predictions completed for loop 65!
LR probs saved for loop 65!


Trying to set attribute `.obs` of view, copying.


computing PCA with n_comps = 65
